In [4]:
import pandas as pd
import re
# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
import spacy
from pprint import pprint
import pyLDAvis
import pyLDAvis.gensim  # don't skip this

C:\Users\axelb\Anaconda3\lib\site-packages\_pytest\mark\structures.py:378: DeprecationWarning: The usage of `cmp` is deprecated and will be removed on or after 2021-06-01.  Please use `eq` and `order` instead.
  @attr.s(cmp=False, hash=False)


In [2]:
import nltk
nltk.download("stopwords")

C:\Users\axelb\Anaconda3\lib\site-packages\nltk\decorators.py:68: DeprecationWarning: `formatargspec` is deprecated since Python 3.5. Use `signature` and the `Signature` object directly
  regargs, varargs, varkwargs, defaults, formatvalue=lambda value: ""
C:\Users\axelb\Anaconda3\lib\site-packages\nltk\lm\counter.py:15: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Sequence, defaultdict
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\axelb\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
# NLTK Stop words
from nltk.corpus import stopwords
stop_words = stopwords.words('english')


In [5]:
data=pd.read_csv('Data/trad_data.csv')
data=data['clean_presentation_trad']

In [6]:

data=[re.sub('\[','',sent) for sent in data]
data=[re.sub('\]','',sent) for sent in data]
data=[re.sub('\"','',sent) for sent in data]
data=[sent.split(',') for sent in data]

<>:1: DeprecationWarning: invalid escape sequence \[
<>:2: DeprecationWarning: invalid escape sequence \]
<>:1: DeprecationWarning: invalid escape sequence \[
<>:2: DeprecationWarning: invalid escape sequence \]
<>:1: DeprecationWarning: invalid escape sequence \[
<>:2: DeprecationWarning: invalid escape sequence \]
<ipython-input-6-a7fc0993f446>:1: DeprecationWarning: invalid escape sequence \[
  data=[re.sub('\[','',sent) for sent in data]
<ipython-input-6-a7fc0993f446>:2: DeprecationWarning: invalid escape sequence \]
  data=[re.sub('\]','',sent) for sent in data]


In [7]:
data=[[re.sub("\'",'',sent) for sent in array] for array in data]

In [8]:
data[0]

['Office',
 ' structure',
 ' production',
 ' contemporary',
 ' art',
 ' exhibitions',
 ' founded',
 ' 2006',
 ' initiative',
 ' exhibitions',
 ' events',
 ' France',
 ' abroad',
 ' presents',
 ' work',
 ' emerging',
 ' artists',
 ' confirmed',
 ' regularly',
 ' collaborates',
 ' guest',
 ' curators',
 ' Office',
 ' THIRTEEN',
 ' shares',
 ' venue',
 ' offices',
 ' Red',
 ' Shoes',
 ' Gallien',
 ' Dejean',
 ' Ali_fib',
 ' gigs']

In [9]:

# Build the bigram and trigram models
bigram = gensim.models.Phrases(data, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
print(trigram_mod[bigram_mod[data][2] ])

['0gms', ' Artist_ Run', ' Space', ' based', ' Sofia', ' Bulgaria', ' goal', ' share', ' experience', ' develop', ' platform', ' art', ' Bulgarian', ' international', ' artists']


In [10]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

install spacy fr to run the below cell in anaconda as administrator : 
pip install https://github.com/explosion/spacy-models/releases/download/fr_core_news_sm-2.0.0/fr_core_news_sm-2.0.0.tar.gz#egg=fr_core_news_sm==2.0.0
pip install https://github.com/explosion/spacy-models/releases/download/fr_core_news_sm-2.0.0/fr_core_news_sm-2.0.0.tar.gz
python -m spacy download fr

In [11]:
# Remove Stop Words
#data_words_nostops = remove_stopwords(data_words)
import spacy
# Form Bigrams
data_words_bigrams = make_bigrams(data)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

#print(stop_words)
data_lemmatized = remove_stopwords(data_lemmatized)

print(data_lemmatized[:1])

NameError: name 'stop_words' is not defined

In [12]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 2), (11, 1), (12, 1), (13, 1), (14, 1), (15, 3), (16, 1), (17, 1), (18, 1), (19, 1), (20, 1), (21, 1), (22, 1), (23, 1)]]


In [13]:
# Human readable format of corpus (term-frequency)
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

[[('abroad', 1),
  ('art', 1),
  ('artist', 1),
  ('collaborate', 1),
  ('confirm', 1),
  ('contemporary', 1),
  ('curator', 1),
  ('dejean', 1),
  ('emerge', 1),
  ('event', 1),
  ('exhibition', 2),
  ('found', 1),
  ('gig', 1),
  ('guest', 1),
  ('initiative', 1),
  ('office', 3),
  ('present', 1),
  ('production', 1),
  ('red', 1),
  ('regularly', 1),
  ('share', 1),
  ('structure', 1),
  ('venue', 1),
  ('work', 1)]]

In [14]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=10, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=33,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [15]:
# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.028*"room" + 0.014*"apartment" + 0.013*"wall" + 0.012*"human" + '
  '0.010*"collector" + 0.010*"number" + 0.010*"building" + 0.009*"temporary" + '
  '0.009*"gallery" + 0.009*"host"'),
 (1,
  '0.023*"organize" + 0.018*"dissemination" + 0.016*"goal" + 0.014*"art" + '
  '0.014*"promotion" + 0.013*"mean" + 0.013*"creation" + 0.013*"expand" + '
  '0.013*"outreach" + 0.013*"territory"'),
 (2,
  '0.043*"residence" + 0.032*"floor" + 0.027*"6b" + 0.020*"workshop" + '
  '0.015*"attention" + 0.014*"industrial" + 0.014*"meter" + 0.013*"sculptor" + '
  '0.013*"special" + 0.012*"century"'),
 (3,
  '0.015*"objective" + 0.010*"late" + 0.007*"business" + 0.004*"neon" + '
  '0.003*"mediate" + 0.001*"eclectic" + 0.000*"implication" + 0.000*"faceted" '
  '+ 0.000*"timely" + 0.000*"broaden"'),
 (4,
  '0.015*"discourse" + 0.013*"specific" + 0.013*"represent" + '
  '0.013*"conceptual" + 0.013*"term" + 0.013*"installation" + 0.011*"movement" '
  '+ 0.011*"commercial" + 0.010*"trade" + 0.009*"museum"

In [16]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -7.307284190926605

Coherence Score:  0.44195118437080605


In [17]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis

C:\Users\axelb\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
6      0.386518 -0.029711       1        1  67.494843
1     -0.011253  0.175071       2        1   8.682828
0     -0.056837  0.048618       3        1   6.490577
4     -0.072263  0.012507       4        1   5.207841
8     -0.076026 -0.265571       5        1   4.893978
5     -0.055773  0.043209       6        1   2.837055
2     -0.042556  0.012311       7        1   2.434788
7     -0.038318  0.003497       8        1   1.761358
3     -0.017706  0.000173       9        1   0.176724
9     -0.015787 -0.000104      10        1   0.020007, topic_info=     Category        Freq           Term       Total  loglift  logprob
2     Default  451.000000         artist  451.000000  30.0000  30.0000
152   Default   65.000000       workshop   65.000000  29.0000  29.0000
1     Default  396.000000            art  396.000000  28.0000  28.0000
93    Default  257.000000          space  257.000000  27.0000  27.0000
37    Default   40.000000         design   40.000000  26.0000  26.0000
125   Default   73.000000        gallery   73.000000  25.0000  25.0000
327   Default   23.000000           room   23.000000  24.0000  24.0000
17    Default   54.000000     production   54.000000  23.0000  23.0000
511   Default   27.000000       organize   27.000000  22.0000  22.0000
154   Default   24.000000       creation   24.000000  21.0000  21.0000
87    Default  161.000000        project  161.000000  20.0000  20.0000
395   Default   22.000000        private   22.000000  19.0000  19.0000
602   Default   13.000000      residence   13.000000  18.0000  18.0000
10    Default  186.000000     exhibition  186.000000  17.0000  17.0000
16    Default   41.000000        present   41.000000  16.0000  16.0000
598   Default   19.000000  dissemination   19.000000  15.0000  15.0000
770   Default   26.000000       building   26.000000  14.0000  14.0000
48    Default   34.000000     experience   34.000000  13.0000  13.0000
23    Default  124.000000           work  124.000000  12.0000  12.0000
5     Default  122.000000   contemporary  122.000000  11.0000  11.0000
446   Default   14.000000          house   14.000000  10.0000  10.0000
1234  Default   11.000000          point   11.000000   9.0000   9.0000
2763  Default   14.000000         pauwel   14.000000   8.0000   8.0000
312   Default   42.000000         center   42.000000   7.0000   7.0000
104   Default   17.000000           goal   17.000000   6.0000   6.0000
981   Default   13.000000           take   13.000000   5.0000   5.0000
652   Default   10.000000          floor   10.000000   4.0000   4.0000
618   Default   15.000000      promotion   15.000000   3.0000   3.0000
131   Default   90.000000       artistic   90.000000   2.0000   2.0000
508   Default   15.000000           mean   15.000000   1.0000   1.0000
...       ...         ...            ...         ...      ...      ...
2783  Topic10    0.000905          villa    4.874574  -0.0747  -7.9316
924   Topic10    0.000905    valencienne    0.747097   1.8009  -7.9316
932   Topic10    0.000905     inhabitant    0.926988   1.5851  -7.9316
931   Topic10    0.000905       informal    2.470672   0.6048  -7.9316
930   Topic10    0.000905      formation    2.301709   0.6757  -7.9316
929   Topic10    0.000905       egyptian    0.926988   1.5851  -7.9316
928   Topic10    0.000905        densely    0.926988   1.5851  -7.9316
927   Topic10    0.000905          civic    0.926988   1.5851  -7.9316
926   Topic10    0.000905      artellewa    1.749474   0.9500  -7.9316
925   Topic10    0.000905     activation    1.342165   1.2150  -7.9316
923   Topic10    0.000905       upstairs    1.034590   1.4753  -7.9316
934   Topic10    0.000905           lewa    1.749474   0.9500  -7.9316
922   Topic10    0.000905           turn    4.510235   0.0030  -7.9316
921   Topic10    0.000905        housing    1.685194   0.9875  -7.9316
920   Topic10    0.000905   headquarters   

In [18]:
pyLDAvis.save_html(vis, 'lda.html')